# Découpage des timecodes en tranches

In [1]:
minute=60*1000
tranche=2*minute
tcDebut=0
ordreGrandeur=1.1

In [2]:
import codecs
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
fSRT="FranceArgentine-DecWhisper-TransGold.srt"
fSRT="Whisper_NouvelleZelande-France-Act2.srt"
fSRT="Whisper_France_Argentine.srt"
fSRT="Whisper_NouvelleZelande-France.srt"

fXLSX=fSRT.replace(".srt",".xlsx")
print (fXLSX)

Whisper_NouvelleZelande-France.xlsx


In [4]:
with codecs.open(fSRT,"r",encoding="utf8") as inFile:
    lignesSRT=[l.strip() for l in inFile.readlines()]
if lignesSRT[0].endswith("1"): lignesSRT[0]="1"
else: print("vérifier le fichier SRT")

In [5]:
def getSeconds(time):
    h, m, s = [float(i) for i in time.replace(",",".").split(':')]
    result=((60*h+m)*60+s)*1000
    return str(int(result))

def getSubtitles(lignes):
    result=[]
    currentState="start"
    currentNum=1
    for l in lignes:
        if currentState=="start" and str(currentNum)==l:
            num=l
            currentState="time"
        elif currentState=="time":
            t1,t2=l.split(" --> ")
            t1=getSeconds(t1)
            t2=getSeconds(t2)
            currentState="text"
        elif currentState=="text":
            text=l
            currentState="skip"
        elif currentState=="skip" and l=="":
            currentState="start"
            currentNum+=1
            result.append({"num":num,"start":t1,"end":t2,"text":text})
    return result        

In [6]:
subtitles=getSubtitles(lignesSRT)
dfSub=pd.DataFrame(subtitles).set_index("num")
dfSub.start=dfSub.start.astype(int)
dfSub.end=dfSub.end.astype(int)

In [7]:
chunks=[]
tcMax=dfSub.end.max()
while tcDebut<tcMax:
    chunkStart=dfSub[dfSub.start>=tcDebut].iloc[0].start
    try:
        chunkEnd=dfSub[dfSub.end>=chunkStart+tranche].iloc[0].end
    except:
        chunkEnd=tcMax
    chunkSize=(chunkEnd-chunkStart)
    print (chunkStart,chunkEnd,chunkSize/1000)
    if tranche/ordreGrandeur<chunkSize<tranche*ordreGrandeur:
        chunks.append((chunkStart,chunkEnd))
    tcDebut=chunkEnd

0 125800 125.8
125800 246120 120.32
246120 366480 120.36
366480 487480 121.0
487480 614880 127.4
614880 739120 124.24
739120 859880 120.76
859880 980880 121.0
980880 1100920 120.04
1100920 1223340 122.42
1223340 1345900 122.56
1345900 1467780 121.88
1467780 1588660 120.88
1588660 1712160 123.5
1712160 1833080 120.92
1833080 1954300 121.22
1954300 2076860 122.56
2076860 2201500 124.64
2201500 2322200 120.7
2322200 2446400 124.2
2446400 2566920 120.52
2566920 2692920 126.0
2692920 2813720 120.8
2813720 2935480 121.76
2935480 3058080 122.6
3058080 3186080 128.0
3186080 3306520 120.44
3306520 3427280 120.76
3427280 3549240 121.96
3549240 3670240 121.0
3670240 3790240 120.0
3790240 3924840 134.6
3924840 4045580 120.74
4045580 4166080 120.5
4166080 4287080 121.0
4287080 4409120 122.04
4409120 4532880 123.76
4532880 4655160 122.28
4655160 4776520 121.36
4776520 4919520 143.0
4919520 5039520 120.0
5039520 5162520 123.0
5162520 5283760 121.24
5283760 5404040 120.28
5404040 5527020 122.98
552702

In [8]:
chunks

[(0, 125800),
 (125800, 246120),
 (246120, 366480),
 (366480, 487480),
 (487480, 614880),
 (614880, 739120),
 (739120, 859880),
 (859880, 980880),
 (980880, 1100920),
 (1100920, 1223340),
 (1223340, 1345900),
 (1345900, 1467780),
 (1467780, 1588660),
 (1588660, 1712160),
 (1712160, 1833080),
 (1833080, 1954300),
 (1954300, 2076860),
 (2076860, 2201500),
 (2201500, 2322200),
 (2322200, 2446400),
 (2446400, 2566920),
 (2566920, 2692920),
 (2692920, 2813720),
 (2813720, 2935480),
 (2935480, 3058080),
 (3058080, 3186080),
 (3186080, 3306520),
 (3306520, 3427280),
 (3427280, 3549240),
 (3549240, 3670240),
 (3670240, 3790240),
 (3924840, 4045580),
 (4045580, 4166080),
 (4166080, 4287080),
 (4287080, 4409120),
 (4409120, 4532880),
 (4532880, 4655160),
 (4655160, 4776520),
 (4919520, 5039520),
 (5039520, 5162520),
 (5162520, 5283760),
 (5283760, 5404040),
 (5404040, 5527020),
 (5527020, 5649660),
 (5649660, 5770500),
 (5770500, 5900300),
 (5900300, 6021260),
 (6021260, 6143100),
 (6143100, 626

In [9]:
dfChunks=pd.DataFrame.from_records(chunks)
dfChunks[0]=pd.to_datetime(dfChunks[0],unit="ms").dt.strftime("%H:%M:%S.%f").str[:-3]
dfChunks[1]=pd.to_datetime(dfChunks[1],unit="ms").dt.strftime("%H:%M:%S.%f").str[:-3]
dfChunks.columns="début fin".split(" ")

In [10]:
dfChunks.to_excel(fXLSX,index=False)